In this section, we want to try out a few experiments with the 2 datasets we have. You will perform some simple prediction tasks from the data, predicting for example the length of a ship, or its future trajectory.

You will do basic modifications on the dataset according to its distribution and its content, and understand how and why the performance of the prediction changes with the changes you made on the dataset.

You will learn how to fix unperfect data to make better predictions.

You will be asked to perform some small machine learning prediction tasks with the data you have. For each task, you will try two ways of dealing with the data and get the performance for both ways.